# 1.1 – Introduction to NumPy and Data Types

Data can come from a wide range of sources and a wide range of formats, including be collections of documents, collections of images, collections of sound clips, collections of numerical measurements, or nearly anything else.
Despite this apparent heterogeneity, it will help us to think of all data fundamentally as arrays of numbers:

- Images–particularly digital images–can be thought of as simply two-dimensional arrays of numbers representing pixel brightness across the area

- Sound clips can be thought of as one-dimensional arrays of intensity versus time.

- Text can be converted in various ways into numerical representations, perhaps binary digits representing the frequency of certain words or pairs of words.

No matter what the data are, the first step in making it analyzable will be to transform them into arrays of numbers. Hence an efficient storage and manipulation of numerical arrays is absolutely fundamental to the process of doing data science.

NumPy (short for *Numerical Python*) provides an efficient interface to store and operate on dense data buffers.
In some ways, NumPy arrays are like Python's built-in ``list`` type, but NumPy arrays provide much more efficient storage and data operations as the arrays grow larger in size.

NumPy arrays form the core of nearly the entire ecosystem of data science tools in Python, so time spent learning to use NumPy effectively will be valuable no matter what aspect of data science interests you.

You can import NumPy and double-check the version:

In [1]:
import numpy
numpy.__version__

'1.21.5'

By convention, most people in the SciPy/PyData world, including us, import NumPy using ``np`` as an alias:

In [2]:
import numpy as np

## Reminder about Built In Documentation

IPython gives you the ability to quickly explore the contents of a package (by using the tab-completion feature)

```ipython
In [3]: np.<TAB>
```
as well as the documentation of various functions (using the ``?`` character)

```ipython
In [4]: np?
```

Try this yourself:

More detailed documentation, along with tutorials and other resources, can be found at http://www.numpy.org.

## Understanding Data Types in Python

Users of Python are often drawn-in by its ease of use, one piece of which is dynamic typing.
While a statically-typed language like C or Java requires each variable to be explicitly declared, a dynamically-typed language like Python skips this specification. For example, in C you might specify a particular operation as follows:

```C
/* C code */
int result = 0;
for(int i=0; i<100; i++){
    result += i;
}
```

While in Python the equivalent operation could be written this way:

```python
# Python code
result = 0
for i in range(100):
    result += i
```

Notice the main difference: in C, the data types of each variable are explicitly declared, while in Python the types are dynamically inferred. This means, for example, that we can assign any kind of data to any variable:

```python
# Python code
x = 4
x = "four"
```

Here we've switched the contents of ``x`` from an integer to a string. The same thing in C would lead (depending on compiler settings) to a compilation error or other unintented consequences:

```C
/* C code */
int x = 4;
x = "four";  // FAILS
```

This sort of flexibility is one piece that makes Python and other dynamically-typed languages convenient and easy to use.
Understanding *how* this works is an important piece of learning to analyze data efficiently and effectively with Python.
But what this type-flexibility also points to is the fact that Python variables are more than just their value; they also contain extra information about the type of the value. We'll explore this more in the sections that follow.

## A Python Integer Is More Than Just an Integer

The standard Python implementation is written in C.
This means that every Python object is simply a cleverly-disguised C structure, which contains not only its value, but other information as well. For example, when we define an integer in Python, such as ``x = 10000``, ``x`` is not just a "raw" integer. It's actually a pointer to a compound C structure, which contains several values.
Looking through the Python 3.4 source code, we find that the integer (long) type definition effectively looks like this (once the C macros are expanded):

```C
struct _longobject {
    long ob_refcnt;
    PyTypeObject *ob_type;
    size_t ob_size;
    long ob_digit[1];
};
```

A single integer in Python 3.4 actually contains four pieces:

- ``ob_refcnt``, a reference count that helps Python silently handle memory allocation and deallocation
- ``ob_type``, which encodes the type of the variable
- ``ob_size``, which specifies the size of the following data members
- ``ob_digit``, which contains the actual integer value that we expect the Python variable to represent.

This means that there is some overhead in storing an integer in Python as compared to an integer in a compiled language like C, as illustrated in the following figure:

![Integer Memory Layout](figures/cint_vs_pyint.png)

Here ``PyObject_HEAD`` is the part of the structure containing the reference count, type code, and other pieces mentioned before.

Notice the difference here: a C integer is essentially a label for a position in memory whose bytes encode an integer value.
A Python integer is a pointer to a position in memory containing all the Python object information, including the bytes that contain the integer value.
This extra information in the Python integer structure is what allows Python to be coded so freely and dynamically.
All this additional information in Python types comes at a cost, however, which becomes especially apparent in structures that combine many of these objects.

## A Python List Is More Than Just a List

Let's consider now what happens when we use a Python data structure that holds many Python objects.
The standard mutable multi-element container in Python is the list.
We can create a list of integers as follows:

In [3]:
L = list(range(10))
L

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [4]:
type(L[0])

int

Or, similarly, a list of strings:

In [5]:
L2 = [str(c) for c in L]
L2

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [6]:
type(L2[0])

str

Because of Python's dynamic typing, we can even create heterogeneous lists:

In [7]:
L3 = [True, "2", 3.0, 4]
[type(item) for item in L3]

[bool, str, float, int]

But this flexibility comes at a cost: to allow these flexible types, each item in the list must contain its own type info, reference count, and other information–that is, each item is a complete Python object.
In the special case that all variables are of the same type, much of this information is redundant: it can be much more efficient to store data in a fixed-type array.
The difference between a dynamic-type list and a fixed-type (NumPy-style) array is illustrated in the following figure:

![Array Memory Layout](figures/array_vs_list.png)

At the implementation level, the array essentially contains a single pointer to one contiguous block of data.
The Python list, on the other hand, contains a pointer to a block of pointers, each of which in turn points to a full Python object like the Python integer we saw earlier.
Again, the advantage of the list is flexibility: because each list element is a full structure containing both data and type information, the list can be filled with data of any desired type.
Fixed-type NumPy-style arrays lack this flexibility, but are much more efficient for storing and manipulating data.

## Fixed-Type Arrays in Python

Python offers several different options for storing data in efficient, fixed-type data buffers.
The built-in ``array`` module (available since Python 3.3) can be used to create dense arrays of a uniform type:

In [8]:
import array
L = list(range(10))
A = array.array('i', L)
A

array('i', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

Here ``'i'`` is a type code indicating the contents are integers.

Much more useful, however, is the ``ndarray`` object of the NumPy package.
While Python's ``array`` object provides efficient storage of array-based data, NumPy adds to this efficient *operations* on that data.
We will explore these operations in later sections; here we'll demonstrate several ways of creating a NumPy array.

## Creating Arrays from Python Lists

First, we can use ``np.array`` to create arrays from Python lists:

In [9]:
# integer array:
np.array([1, 4, 2, 5, 3])

array([1, 4, 2, 5, 3])

Remember that unlike Python lists, NumPy is constrained to arrays that all contain the same type.
If types do not match, NumPy will upcast if possible (here, integers are up-cast to floating point):

In [10]:
np.array([3.14, 4, 2, 3])

array([3.14, 4.  , 2.  , 3.  ])

If we want to explicitly set the data type of the resulting array, we can use the ``dtype`` keyword:

In [11]:
np.array([1, 2, 3, 4], dtype='float32')

array([1., 2., 3., 4.], dtype=float32)

Finally, unlike Python lists, NumPy arrays can explicitly be multi-dimensional; here's one way of initializing a multidimensional array using a list of lists:

In [12]:
# nested lists result in multi-dimensional arrays
np.array([range(i, i + 3) for i in [2, 4, 6]])

array([[2, 3, 4],
       [4, 5, 6],
       [6, 7, 8]])

The inner lists are treated as rows of the resulting two-dimensional array.

## Creating Arrays from Scratch

Especially for larger arrays, it is more efficient to create arrays from scratch using routines built into NumPy.
Here are several examples:

In [13]:
# Create a length-10 integer array filled with zeros
np.zeros(10, dtype=int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [14]:
# Create a 3x5 floating-point array filled with ones
np.ones((3, 5), dtype=float)

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [15]:
# Create a 3x5 array filled with 3.14
np.full((3, 5), 3.14)

array([[3.14, 3.14, 3.14, 3.14, 3.14],
       [3.14, 3.14, 3.14, 3.14, 3.14],
       [3.14, 3.14, 3.14, 3.14, 3.14]])

In [16]:
# Create an array filled with a linear sequence
# Starting at 0, ending at 20 (not including 20), stepping by 2
# (this is similar to the built-in range() function)
np.arange(0, 20, 2)

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])

In [17]:
# Create an array of five values evenly spaced between 0 and 1
np.linspace(0, 1, 5)

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

In [18]:
# Create a 3x3 array of uniformly distributed
# random values between 0 and 1
np.random.random((3, 3))

array([[0.83354788, 0.90198506, 0.90713595],
       [0.27610358, 0.20223703, 0.53876044],
       [0.60565376, 0.72049508, 0.24446639]])

In [19]:
# Create a 3x3 array of normally distributed random values
# with mean 0 and standard deviation 1
np.random.normal(0, 1, (3, 3))

array([[ 0.86331557, -1.40260004, -0.49222931],
       [ 1.93132436, -1.15519862,  0.15907427],
       [-0.02812873, -0.44618752,  0.53310976]])

In [20]:
# Create a 3x3 array of random integers in the interval [0, 10)
np.random.randint(0, 10, (3, 3))

array([[8, 4, 0],
       [9, 6, 8],
       [9, 4, 9]])

In [21]:
# Create a 3x3 identity matrix
np.eye(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [22]:
# Create an uninitialized array of three integers
# The values will be whatever happens to already exist at that memory location
np.empty(3)

array([1., 1., 1.])

## NumPy Standard Data Types

NumPy arrays contain values of a single type, so it is important to have detailed knowledge of those types and their limitations.
Because NumPy is built in C, the types will be familiar to users of C, Fortran, and other related languages.

The standard NumPy data types are listed in the following table.
Note that when constructing an array, they can be specified using a string:

```python
np.zeros(10, dtype='int16')
```

Or using the associated NumPy object:

```python
np.zeros(10, dtype=np.int16)
```

| Data type	    | Description |
|---------------|-------------|
| ``bool_``     | Boolean (True or False) stored as a byte |
| ``int_``      | Default integer type (same as C ``long``; normally either ``int64`` or ``int32``)| 
| ``intc``      | Identical to C ``int`` (normally ``int32`` or ``int64``)| 
| ``intp``      | Integer used for indexing (same as C ``ssize_t``; normally either ``int32`` or ``int64``)| 
| ``int8``      | Byte (-128 to 127)| 
| ``int16``     | Integer (-32768 to 32767)|
| ``int32``     | Integer (-2147483648 to 2147483647)|
| ``int64``     | Integer (-9223372036854775808 to 9223372036854775807)| 
| ``uint8``     | Unsigned integer (0 to 255)| 
| ``uint16``    | Unsigned integer (0 to 65535)| 
| ``uint32``    | Unsigned integer (0 to 4294967295)| 
| ``uint64``    | Unsigned integer (0 to 18446744073709551615)| 
| ``float_``    | Shorthand for ``float64``.| 
| ``float16``   | Half precision float: sign bit, 5 bits exponent, 10 bits mantissa| 
| ``float32``   | Single precision float: sign bit, 8 bits exponent, 23 bits mantissa| 
| ``float64``   | Double precision float: sign bit, 11 bits exponent, 52 bits mantissa| 
| ``complex_``  | Shorthand for ``complex128``.| 
| ``complex64`` | Complex number, represented by two 32-bit floats| 
| ``complex128``| Complex number, represented by two 64-bit floats| 

## Reading Array from a Text File

The simplest way of reading an array from a text file is to use ``np.loadtxt()``. This method is used when the data has no missing or corrupt values. Such a data is called *clean*.

Suppose your data is stored in a comma-separated values (csv) file ``data/array_float.csv``:

In [23]:
!head data/array_float.csv # inspect the file

74.419,75.892,56.583,96.165
30.644,3.657,67.424,72.027
23.227,71.508,77.222,80.140


To read this file use into a NumPy array use: 

In [24]:
A = np.loadtxt("data/array_float.csv", delimiter = ",") # the default option is delimiter=None and dtype=float
A

array([[74.419, 75.892, 56.583, 96.165],
       [30.644,  3.657, 67.424, 72.027],
       [23.227, 71.508, 77.222, 80.14 ]])

If the data is of the integer type, you should add an appropriate key:

In [25]:
!head data/array_int.csv # inspect the file

  1,  2,  3,  4
  5,  6,  7,  8
  9, 10, 11, 12


In [26]:
B = np.loadtxt("data/array_int.csv", delimiter = ",", dtype=int)
B

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12]])

Note that trying to read a float data with the ``dtype=int`` key will yield an error:

In [27]:
np.loadtxt("data/array_float.csv", delimiter = ",", dtype=int)

ValueError: invalid literal for int() with base 10: '74.419'

To save an array to a text file, use ``np.savetxt()``:

In [28]:
np.savetxt("data/my_array.csv", B, fmt="%3i", delimiter=",")

The key `fmt` is best explained via examples:

- ``fmt="%1i"`` gives
```
1,2,3,4
5,6,7,8
9,10,11,12
```

- ``fmt="%3i"`` gives
```
  1,  2,  3,  4
  5,  6,  7,  8
  9, 10, 11, 12
```

- ``fmt="%5i"`` gives
```
    1,    2,    3,    4
    5,    6,    7,    8
    9,   10,   11,   12
```

- ``fmt="%5.1f"`` gives
```
  1.0,  2.0,  3.0,  4.0
  5.0,  6.0,  7.0,  8.0
  9.0, 10.0, 11.0, 12.0
```

- ``fmt="%5.2f"`` gives
```
 1.00, 2.00, 3.00, 4.00
 5.00, 6.00, 7.00, 8.00
 9.00,10.00,11.00,12.00
```

More information on NumPy Input and Output routines can be found on the [NumPy.IO](https://numpy.org/doc/stable/reference/routines.io.html) documentation website.

## Exercises

**Exercise 1.1.1** Create a vector of length 10 with integer values evenly distributed between 5 and 50.  Do this in two different ways. <br> The wanted output is ``[ 5 10 15 20 25 30 35 40 45 50]``.

In [1]:
# write your solution here



**Exercise 1.1.2** Create a vector of even integers from 10 to 30 (including 30). Do this in two different ways. <br> The wanted output is ```[10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]```.

In [2]:
# write your solution here



**Exercise 1.1.3** Create a $3\times 3$ identity matrix in two different ways. <br>
The wanted outcome is:
```
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
```

In [3]:
# write your solution here



**Exercise 1.1.4** Create a $2\times 4$ array of zeros.

In [4]:
# write your solution here



**Exercise 1.1.5** Create a $4\times3$ floating-point array of ones.

In [5]:
# write your solution here



**Exercise 1.1.6** Create a $5\times5$ zero matrix with elements on the main diagonal equal to 1, 2, 3, 4, 5.

In [6]:
# write your solution here



**Exercise 1.1.7** Create the following pattern without hardcoding: ```[1, 1, 1, 2, 2, 2, 3, 3, 3]```

In [7]:
# write your solution here



**Exercise 1.1.8** Create a $4\times4$ array of random integers in the interval [-5, 5]

In [8]:
# write your solution here



**Exercise 1.1.9** Execute the code below and explain the result: why 1, 2, 3 became "1", "2", "3"?

In [37]:
np.array([["a","b","c"],[1,2,3]])

array([['a', 'b', 'c'],
       ['1', '2', '3']], dtype='<U21')

In [36]:
# write your solution here

---

<!--NAVIGATION-->
< [Contents](../index.ipynb) | [1.2 – The Basics of NumPy Arrays](L12_The_Basics_Of_NumPy_Arrays.ipynb) >


*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; also available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*